For each mouse
1. (Manual) Find summary excel file, extract names of green and yellow sessions
2. VidsDCIMG = Find video files, one for each trial
3. refImg = Find reference image
4. trialTypes = Find trial types
5. t1.mat, t2.mat = Find warping files 
6. allenBrain = Find allen brain map

Processing:
1. Testing_A:
    * For each mouse, load ::: refImg, t1.mat, allenBrain
      * Gallero\SDT-TDT\overlay\mou_9
        - L_modified (brain areas for all mice) ::: Gallero\SDT-TDT\overlay\mou_9/L_modified.mat
        - RefImg ::: Gallero\SDT-TDT\overlay\mou_9\refImg_ROIs.mat
    * For each session, load ::: t2.mat
      - Gallero/mou_9/TDT/20180315/widefield_labview/ROI_Allen/registration_transform_1510.mat
    * Apply transform to refImg, overplot allenBrain, check that they align
2. Testing_B:
    * Load all trial videos
        - Gallero\mou_9\TDT\20180301\widefield_labview
    * Compute time-average images
    * Apply same as Testing_A
    * Plot a few random sessions
3. Process real data - for each session:
    * Provide list of sessions to analyze
    * Provide root folder to mouse data
    * Provide root folder to overlay data
    * Videos
        * Load all trial videos
        * Apply transformation
        * Apply allen map to extract activities, average over pixels
        * Store in HDF5
    * Trial Intervals:
        * Load trial interval file
          - Gallero\mou_9\TDT\20180301\widefield_labview\a\Matt_files\20180301a.mat
        * Store 3rd column to HDF5
    * Trial Types:
        * Load trial type file
          - \Gallero\mou_9\TDT\20180301\widefield_labview\a\Matt_files/trials_ind.mat
        * Parse, store to HDF5
4. Postprocess datatypes
    * Read HDF5
    * Merge trials for each session
    * Calc times from intervals
    * Apply polyfit, look
    * Perform DFF, store


Notes:

* Matlab Fitting routines
   - t1= align to mapping day (functional); aligns every day to a common day.
   - t2= aligns the common day to Allen institue

`
tform = fitgeotrans(movingPoints,fixedPoints,'polynomial',3);
Jregistered = imwarp(refImg*7000,tform,'OutputView',imref2d(size(A_scaled))); %aligned map
`

**TODO**:
* Compare TrialTypes to to trial_idx.m where applicable, otherwise trial_error.m
* Compare resulting bn_trial with what Yasir got

In [10]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fpath

from lib.gallerosalas.preprocess_raw import preprocess
import lib.gallerosalas.preprocess_common as prepcommon


%load_ext autoreload
%autoreload 2

Appended root directory /media/aleksejs/DataHDD/work/codes/comp-neuro/analysis-mesoscopic/pub-2020-exploratory-analysis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
FPS = 20  # Hz
mice = ['mou_9', 'mou_6', 'mou_7', 'mou_5']
extra_mice = ['mou_15_Ariel', 'mou_18_Ariel']

In [3]:
# pathTGT = gui_fpath('root path to TGT')
pathTGT = '/run/user/1000/gvfs/smb-share:server=130.60.51.15,share=neurophysiology-storage2/Gallero'

In [4]:
# pathOverlay = gui_fpath('root path to overlay')
pathOverlay = '/run/user/1000/gvfs/smb-share:server=130.60.51.15,share=neurophysiology-storage2/Gallero/SDT-TDT/overlay'

In [5]:
# pathPreferences = gui_fpath('root path to overlay')
# pathPreferences = '/media/alyosha/Data/TE_data/yasirdata_raw'
pathPreferences = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [ ]:
pathDict = {
    'TGT' : pathTGT,
    'Overlay' : pathOverlay,
    'Preferences' : pathPreferences
}

In [ ]:
prep = preprocess(pathDict)

# 1. Test transform on Ref files

In [ ]:
for mouseName, pathRef in prep.pathRef.items():
    img = prep.load_ref_img(pathRef)
    t2 = prep.load_t2(prep.pathT2[mouseName])
    print(mouseName)
    
    imgT2 = prep.transform_img(img, t2)[0]
    prep.plot_transforms(img, imgT2, imgT1=None)

# 2. Test transform on Video Files

In [ ]:
%%time
for mousename in sorted(set(prep.dataPaths['mouse'])):
    print(mousename)
    prep.test_transform_vids(mousename)

# 3. Pool-process video files

In [ ]:
prep.process_video_files('mou_5', skipExisting=False)

## 3.1 Test if result is sensible

In [ ]:
import h5py 
rez = []
for idx, row in prep.dataPaths.iterrows():
    session = row['day'] + '_' + row['session']
    fpath = os.path.join(pathPreferences, row['mouse'] + '.h5')    
    
    with h5py.File(fpath, 'r') as f:
        dataRSP = np.copy(f['data'][session])
        rez += [np.nanmean(dataRSP, axis=(0, 1))]
        
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(np.array(rez), columns=np.arange(27)))

# 4. Pull trial types and times

* Also compute d'

In [ ]:
import h5py

for mouseName in mice:
    with h5py.File(os.path.join(pathPreferences, mouseName + '.h5'), 'a') as f:
        del f['metadata']
        del f['accuracy']
        del f['dprime']

In [11]:
prep.process_metadata_files(pathPreferences)

NameError: name 'prep' is not defined

In [ ]:
for idx, row in prep.dataPaths.iterrows():
    session = row['day'] + '_' + row['session']
    fpath = os.path.join(pathPreferences, row['mouse'] + '.h5')
    df = pd.read_hdf(fpath, '/metadata/' + session)
    display(df.head())

# 5. Find and add delay times

In [ ]:
prep.get_append_delay_times(pathPreferences)

# 6. Baseline subtraction

# 6.1 Plot stitched session, fit poly

**TODO**:
* Purge mismatching trial counts for trialStruct vs vid:
    - Go over data, delete last
* Very high variance of individual consecutive trials. May render entire idea useless
  - Explore more channels, mice, naive vs expert
  - Compare if Yaro has the same problem
  - Ask Yasir what he thinks about the origin of these fluctuations:
      - Real?
      - Motion artifacts?
      - Sth else?

In [ ]:
%matplotlib notebook
for idx, row in prep.dataPaths.iterrows():
    try: 
#         session = row['day'] + '_' + row['session']
        session="2017_03_29_session01"
        print(row['mouse'], session)
        
        times, dataRSP = prep.get_pooled_data_rel_times(pathPreferences, row['mouse'], session)
        prepcommon.example_poly_fit(times, dataRSP, ord=50, iCh=6)
    except:
        print('-- Failed')
    break

# 6.2 Process all sessions

In [ ]:
FPS = 20
iMin = 0 * FPS
iMax = 1 * FPS

In [ ]:
prep.crop_mismatch_trials(pathPreferences)

In [ ]:
prep.baseline_subtraction_dff(pathPreferences, iMin, iMax, skipExist=True)

In [ ]:
prep.baseline_subtraction_poly(pathPreferences, ord=50, skipExist=True)

# 7. Purification

### 7.1 Drop bad sessions

- mou_7 20171129_c
- mou_6 20171006_a

In [ ]:
prep.drop_preprocess_session(pathPreferences, 'mou_6', '20171006_a')

In [ ]:
prep.drop_preprocess_session(pathPreferences, 'mou_7', '20171129_c')

# 8. Append Active/Passive

In [6]:
import h5py

In [9]:
for mousename in mice:
    pathH5 = os.path.join(pathPreferences, mousename + '.h5')
    with h5py.File(pathH5, 'r') as f:
        sessions = list(f['metadata'].keys())
        
    for session in sessions:
        df = pd.read_hdf(pathH5, '/metadata/'+session)
        display(df)
        break
    break

,time_stamp,stimulus_time,decision_time,reward_time,end_time,trialType
0,09:40:11.3012,5858.6,11056.1,NaN,14697.5,FA
1,09:40:30.1401,2895.3,9642.9,NaN,12383.2,CR
2,09:40:46.6537,2869.3,9594.0,NaN,12075.2,Miss
3,09:41:02.8543,2874.3,7988.8,NaN,11675.2,FA
4,09:41:18.6589,2872.3,9596,NaN,12321.3,CR
...,...,...,...,...,...,...
179,10:27:10.8109,2876.0,9602,NaN,12377.0,CR
180,10:27:27.3109,2874.0,9621.0,NaN,12363.0,CR
181,10:27:43.7999,2876.0,7947.0,7943.0,10581.0,Hit
182,10:27:58.5089,2869.0,9593.0,NaN,12311.0,CR


In [ ]:

for mouseName, dfMouse in self.dfSession.groupby(['mousename']):
    pathMouse = os.path.join(pathTDT, mouseName)
    prepcommon._testdir(pathMouse)

    for dayName, dfDay in dfMouse.groupby(['dateKey']):
        print(mouseName, dayName)
        pathDay = os.path.join(pathMouse, dayName, 'widefield_labview')
        pathAllen = os.path.join(pathDay, 'ROI_Allen', 'registration_transform_1510.mat')
        haveDir = prepcommon._testdir(pathDay, critical=False)
        haveFile = prepcommon._testfile(pathAllen, critical=False)

        if haveDir and haveFile:
            self.pathT1[mouseName][dayName] = pathAllen
            for idx, row in dfDay.iterrows():
                sessionSuffix = row['sessionKey']

                pathSession = os.path.join(pathDay, sessionSuffix)
                pathMat = os.path.join(pathSession, 'Matt_files')
                # pathTrialInd = os.path.join(pathMat, 'trials_ind.mat')

                matFiles = [f for f in os.listdir(pathMat) if os.path.splitext(f)[1] == '.mat' and f[:9] == 'Behavior_']
                assert len(matFiles) == 1
                pathTrialStruct = os.path.join(pathMat, matFiles[0])

                pathActivePassive = os.path.join(pathMat, 'trials_with_and_wo_initial_moves_OCIA_from_movie.mat')

                prepcommon._testdir(pathSession)
                prepcommon._testdir(pathMat)
                # prepcommon._testfile(pathTrialInd)
                prepcommon._testfile(pathTrialStruct)
                prepcommon._testfile(pathActivePassive)

                self.dataPaths = pd_append_row(self.dataPaths, [
                    mouseName, dayName, sessionSuffix, pathSession, #pathTrialInd,
                    pathTrialStruct,
                    pathActivePassive
                ])